In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter

from utils import fill_doors, door_reveal, identify_result, random_choice

In [2]:
# set number of doors and create a list of n doors
n = 3
doorlist = list(np.arange(1, n+1))

# set number of iterations
N = 1000

### Part 1: Set-up

#### i. Initiate all doors
Create a dataframe that has N sets of n doors, each door with either a goat or a car behind it.

In [3]:
# create a dataframe with 1 set of doors
df = fill_doors(n)

# loop over the sample size and fill a set of doors for each sample
for i in np.arange(0, N):

    # fill the new doors
    doors = fill_doors(n)
    
    # append the set of doors to the dataframe
    df = pd.concat([df, doors]).reset_index(drop=True)


#### ii. First pick

Randomly pick the first door that you open for each simulation.

In [4]:
# pick your first door randomly
df["first_door"] = df.apply(lambda row: random.choice(list(doorlist)), axis=1)

# determine what prize was behind that first door
df["first_choice"] = df.apply(lambda row: row[row["first_door"]], axis=1)

#### iii. Door reveal
Monty Hall reveals what is behind one of the two doors you did not pick.

In [5]:
# identify which door will be revealed, based on your first choice
df["door_revealed"] = df.apply(lambda row: door_reveal(row, doorlist), axis=1)

### Part 2a: Classic Monty Hall set-up

#### iv. Switch or stay
Randomly choose to switch or stay.

In [6]:
# randomly choose switch or stay
df["decision"] = df.apply(lambda row: random.choice(["switch", "stay"]), axis=1)

#### v. Determine the outcomes
Determine what percent of the time you won by switching vs. staying.

In [7]:
# determine the outcomes
df["outcome"] = df.apply(lambda row: identify_result(row), axis=1)

# probability for switching
switch_outcomes = Counter(df.loc[((df["decision"]=="switch")), "outcome"])
switch_win_p = switch_outcomes["win"] / (switch_outcomes["win"] + switch_outcomes["lose"])
switch_win_p = np.round(switch_win_p * 100, 1)

# probability for staying
stay_outcomes = Counter(df.loc[((df["decision"]=="stay")), "outcome"])
stay_win_p = stay_outcomes["win"] / (stay_outcomes["win"] + stay_outcomes["lose"])
stay_win_p = np.round(stay_win_p * 100, 2)


##### Print the result

In [8]:
print(f"{N} iterations: switching won {switch_win_p}%, staying won {stay_win_p}%.")

1000 iterations: switching won 67.2%, staying won 32.55%.


### Part 2b: Randomize the door choice (Alternative method)

#### iv. Randomly pick a door
After the reveal, the 2 remainig doors have been randomized such that you lose all prior information you had from your first choice. Now, you simply randomly select from the two remaining doors.

In [9]:
# make the random choice
df["random_choice"] = df["door_revealed"].apply(lambda x: random_choice(x, doorlist))

#### v. Determine the outcomes

In [10]:
# determine the winner
df["rc_outcome"] = df.apply(lambda row: row[row["random_choice"]], axis=1)
df["rc_outcome"] = df["rc_outcome"].replace({"g": "lose", "c": "win"})

# probabilities
rc_outcomes = Counter(df["rc_outcome"])
rc_win_p = rc_outcomes["win"] / (rc_outcomes["win"] + rc_outcomes["lose"])
rc_win_p = np.round(rc_win_p * 100, 2)

##### Print the result

In [11]:
print(f"{N} iterations: random choice gives you a win {rc_win_p}%.")

1000 iterations: random choice gives you a win 50.75%.
